In [1]:
import pandas as pd
import numpy as np
import pyodbc 
import time
import datetime
import sqlite3
import os
from sqlalchemy import create_engine
from pandas.tseries.offsets import DateOffset

In [2]:
import sys
import logging
path = r'..\..\Logs\clinical_log.log'
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("NOMS-Survey Completion")
# logger.info("testing log")

## DB Query

In [3]:
conn = create_engine(r'mssql+pyodbc://@PYTHONSERVER\SQLEXPRESS/InSync?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes', fast_executemany=True)

# conn = sqlite3.connect('../../InSyncConnection/Database/InSyncClinical.db')
# cursor = conn.cursor()

# # # printing all table names
# sql_query = """SELECT name FROM sqlite_master
#      WHERE type='table';"""

# cursor.execute(sql_query)
# print(cursor.fetchall())

### Encounter Query 

In [4]:
try:
    # Select all patients from EcounterSummary Table
    testPatients = '''
    SELECT
        PatientId
    FROM 
        emr_PatientDetails
    WHERE 
        LOWER(FirstName) LIKE '%test%'
        OR LOWER(LastName) LIKE '%test%'
        OR LOWER(FirstName) LIKE '%patient%'
        OR LOWER(LastName) LIKE '%patient%'
        or CAST(MRNNumber AS INTEGER) < 55
    '''
    sql=f'''
    SELECT 
        PatientId as PatientID,
        VisitDateTime as EncounterDate
    FROM 
        tblEncounterSummary
        LEFT JOIN tblENcounterType ON (tblEncounterSummary.EncounterTypeId=tblENcounterType.EncounterTypeId)
    WHERE 
        IsBillable = 'TRUE'
        AND PatientID NOT IN ({testPatients})
    ORDER BY 
        PatientID
    '''
    encounter_df = pd.read_sql(sql, conn)
    encounter_df['EncounterDate'] = pd.to_datetime(encounter_df['EncounterDate'])
    
    #filter out encounters before 3/1 and keep the first encounter date
    encounter_filter = encounter_df['EncounterDate'] >= '2023-03-01'
    filtered_dates = encounter_df[encounter_filter].copy()
    filtered_dates.sort_values(by=['PatientID', 'EncounterDate'], inplace=True)
    encounter_df = filtered_dates.drop_duplicates(subset="PatientID", keep='first').copy()
    
    # get encounter time by hour
    encounter_df['Hour'] = encounter_df["EncounterDate"].dt.floor('h')
    
    # needed for filtering out transfer patients
    encounter_dictionary = dict(zip(encounter_df.PatientID, encounter_df.EncounterDate))
    
    logger.info(f"Successfully queried tblEncounterSummary.")
except Exception as e:
    logger.error(f"Failed to query tblEncounterSummary.", exc_info=True) 
    print(e)
    sys.exit(1)
encounter_df

,PatientID,EncounterDate,Hour
0,620285,2024-03-20 19:15:00,2024-03-20 19:00:00
10,620288,2023-10-08 12:00:00,2023-10-08 12:00:00
116,620292,2023-07-05 17:00:00,2023-07-05 17:00:00
158,620293,2023-10-10 18:15:00,2023-10-10 18:00:00
316,620295,2023-03-05 13:00:00,2023-03-05 13:00:00
...,...,...,...
326103,630230,2025-01-21 14:00:00,2025-01-21 14:00:00
326104,630232,2025-01-24 11:00:00,2025-01-24 11:00:00
326105,630240,2025-01-26 14:00:00,2025-01-26 14:00:00
326106,630241,2025-01-24 09:15:00,2025-01-24 09:00:00


### PatientDetails Query

In [5]:
try:
    # Select all patients
    sql= f'''
    SELECT 
        DISTINCT (PatientID) AS PatientID,
        FirstName,
        LastName,
        DOB,
        MRNNumber,
        PhoneNo AS Phone
    FROM 
        emr_PatientDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY 
        PatientID
    '''
    patient_details_df = pd.read_sql(sql, conn)
    
    # filter out MRN != XXX2
    MRN_filter = patient_details_df['MRNNumber'].map(lambda value: True if value[-1] == "2" else False)
    patient_details_df = patient_details_df[MRN_filter].copy()
    
    # filter out test patients
    test_filter = (patient_details_df['LastName'] != "Test") & (patient_details_df["FirstName"] != "Test")
    patient_details_df = patient_details_df[test_filter].copy()
    
    logger.info(f"Successfully queried emr_PatientDetails.")
except Exception as e:
    logger.error(f"Failed to query emr_PatientDetails.", exc_info=True) 
    print(e)
    sys.exit(1)

patient_details_df[patient_details_df['PatientID'].duplicated()]

,PatientID,FirstName,LastName,DOB,MRNNumber,Phone


### Discharged Query 

In [6]:
try:
    # Select all patients
    sql = f'''
    SELECT 
        PatientID,
        finalEncounterDate AS 'Final Encounter',
        is_ActiveInInSync,
        is_lastEncounterDischarge
    FROM 
        ptPatient_Activity
    WHERE
        PatientID NOT IN ({testPatients})
    '''
    disharge_df = pd.read_sql(sql, conn)
    
    # filter out active patients
    discharged_patients_mask = (disharge_df['is_ActiveInInSync'] == 0) | (disharge_df['is_lastEncounterDischarge'] == 1)
    discharged_patients_df = disharge_df[discharged_patients_mask].copy()
    
    # create discharge status
    discharged_patients_df['Status'] = "Discharged"
    
    discharged_patients_df.drop(columns=['is_ActiveInInSync','is_lastEncounterDischarge'],
                                axis=1,
                                inplace=True)
    discharged_patients_df['Final Encounter'] = pd.to_datetime(discharged_patients_df['Final Encounter'])
    
    logger.info(f"Successfully queried ptPatient_Activity.")
except Exception as e:
    logger.error(f"Failed to query ptPatient_Activity.", exc_info=True) 
    print(e)
    sys.exit(1)
discharged_patients_df

,PatientID,Final Encounter,Status
0,620285,2024-05-21 11:15:00,Discharged
2,620290,2023-02-06 12:53:00,Discharged
3,620291,2022-07-07 12:27:00,Discharged
7,620296,2022-05-23 13:33:00,Discharged
8,620298,2024-09-13 09:00:00,Discharged
...,...,...,...
7150,629751,2024-12-18 14:30:00,Discharged
7171,629777,2024-12-19 17:00:00,Discharged
7267,629903,2024-12-23 09:00:00,Discharged
7292,629933,2025-01-22 12:11:00,Discharged


### NOMS Query 

#### Baseline NOMS Taken

In [7]:
try:
    sql=f'''
    SELECT 
        PatientID,
        CreatedOn,
        StatusDesc AS 'Baseline NOMS'
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE '%Baseline%'
        AND (StatusDesc LIKE '%Entered%'
        OR StatusDesc LIKE '%Completed%')
        AND
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    df = pd.read_sql(sql, conn)
    df['CreatedOn'] = pd.to_datetime(df['CreatedOn']).dt.date
    
    # filter before march
    march_mask = datetime.date(2023,3,1)
    baseline_df = df[df['CreatedOn'] >= march_mask].copy()
    
    # Prettify data for mergers later
    baseline_df['Baseline NOMS'] = baseline_df['Baseline NOMS'].map(lambda status: True if str(status) in ["Entered Into SPARS"] else False)
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Baselines.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Baselines.", exc_info=True) 
    print(e)
    sys.exit(1)

baseline_df

,PatientID,CreatedOn,Baseline NOMS
0,620331,2024-04-25,True
1,620381,2023-07-31,True
2,620421,2023-05-03,True
3,620431,2023-04-25,True
4,620461,2023-05-11,True
...,...,...,...
532,630048,2025-01-13,True
533,630058,2025-01-20,True
534,630068,2025-01-14,True
535,630098,2025-01-15,True


#### Reassessment NOMS

In [8]:
try:
    sql=f'''
    SELECT 
        PatientID,
        CreatedOn
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE 'Reassessment%'
        AND (StatusDesc LIKE '%Entered%'
        OR StatusDesc LIKE '%Completed%')
        AND
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    df = pd.read_sql(sql, conn)
    df['CreatedOn'] = pd.to_datetime(df['CreatedOn']).dt.date
    
    # filter before march
    march_mask = datetime.date(2023,3,1)
    reassessment_df = df[df['CreatedOn'] >= march_mask]
    
#     # find transfer patients
#     transfer_patients=[]
#     for patient in reassessment_df['PatientID']:
#         if False != encounter_dictionary.get(patient, False):
#             transfer_patients.append(patient)
            
#     # remove transfers if reassessment is before 6 months
#     earliest_reassessment_date = datetime.date(2023,8,1)
#     for patient in transfer_patients:
#         first_encounter = encounter_dictionary.get(patient)
#     #     incremented_first_encounter = first_encounter + np.timedelta64(6, 'M')
#         reassmessnent_date = df[df['PatientID'] == patient]['CreatedOn']
#         print(reassmessnent_date)
#         if (reassmessnent_date.values > earliest_reassessment_date) == False:
#             reassessment_df = reassessment_df[reassessment_df['PatientID'] != patient].copy()
    
    # Prettify data for megers later
    reassessment_df = reassessment_df[['PatientID']]
    reassessment_df['6 Month Reassessment NOMS'] = True
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Reassessments.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Reassessments.", exc_info=True) 
    print(e)
    sys.exit(1)

#### Discharge NOMS

In [9]:
try:
    sql=f'''
    SELECT 
        DISTINCT (PatientID),
        CreatedOn AS 'Discharge Date',
        AssessmentKey AS 'Discharged'
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE '%Discharge%'
        AND (StatusDesc LIKE '%Entered%'
        OR StatusDesc LIKE '%Completed%')
        AND
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    df = pd.read_sql(sql, conn)
    df['Discharge Date'] = pd.to_datetime(df['Discharge Date']).dt.date
    
    # filter before march
    march_mask = datetime.date(2023,3,1)
    discharge_df = df[df['Discharge Date'] >= march_mask]
    
    
    # Prettify data for megers later
    discharge_df = discharge_df[['PatientID']]
    discharge_df.drop_duplicates(inplace=True)
    discharge_df['Discharge NOMS'] = True

    logger.info(f"Successfully queried tblNOMS_AllAssessments for Discharges.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Discharges.", exc_info=True) 
    print(e)
    sys.exit(1)
    
discharge_df

,PatientID,Discharge NOMS
0,620501,True
1,620541,True
2,620581,True
3,620671,True
4,620681,True
...,...,...
252,628958,True
253,629068,True
254,629128,True
255,629148,True


#### NOMS Refusals

In [10]:
try:
    sql=f'''
    SELECT 
        PatientID,
        InterviewConductedNoID AS Reason
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE 'Base%'      AND
        InterviewConductedID LIKE '%0%' AND
        PatientID NOT IN ({testPatients})
    '''
    refused_df = pd.read_sql(sql, conn)
    refused_df['Consent'] = 'No'
    # fill in blank reasons with 'None Given'
    refused_df = refused_df.replace(r'^\s*$', 'None Given', regex=True)

    # Reorder columns
    refused_df = refused_df[['PatientID',
                             'Consent',
                             'Reason']]
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Baseline Refusals.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Baseline Refusals.", exc_info=True) 
    print(e)
    sys.exit(1)
refused_df

,PatientID,Consent,Reason
0,621243,No,3 - Client Refused This Interview
1,622991,No,3 - Client Refused This Interview
2,621451,No,4 - Client Was Not Reached For Interview
3,621171,No,2 - Client Was Impaired Or Unable To Provide C...
4,622681,No,3 - Client Refused This Interview
...,...,...,...
448,621882,No,None
449,621540,No,None
450,624222,No,3 - Client Refused This Interview
451,620575,No,None


#### NOMS Consents

In [11]:
try:
    sql=f'''
    SELECT 
        PatientID
    FROM 
        tblNOMS_AllAssessments
    WHERE 
        AssessmentKey LIKE 'Base%'      AND
        InterviewConductedID LIKE '%1%' AND
        PatientID NOT IN ({testPatients})
    '''
    agreed_df = pd.read_sql(sql, conn)
    agreed_df
    # add Consent
    agreed_df["Consent"] = 'Yes'
    
    logger.info(f"Successfully queried tblNOMS_AllAssessments for Baseline Consents.")
except Exception as e:
    logger.error(f"Failed to query tblNOMS_AllAssessments for Baseline Consents.", exc_info=True) 
    print(e)
    sys.exit(1)
agreed_df

,PatientID,Consent
0,628878,Yes
1,629448,Yes
2,625535,Yes
3,626485,Yes
4,622240,Yes
...,...,...
393,627707,Yes
394,620966,Yes
395,626135,Yes
396,622248,Yes


#### Append Consent dfs 

In [12]:
consent_df = pd.concat([refused_df, agreed_df], ignore_index=True)

### Entered into Spars

#### Baselines

In [13]:
try:
    sql=f'''
    SELECT 
        PatientID,
        StatusDesc AS 'Baseline SPARS'
    FROM 
        tblNOMSAssessmentDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    baseline_SPARS = pd.read_sql(sql,conn)
        
    logger.info(f"Successfully queried tblNOMSAssessmentDetails for Baselines entered to SPARS.")
except Exception as e:
    logger.error(f"Failed to query tblNOMSAssessmentDetails for Baselines entered to SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)

#### Reassessments

In [14]:
try:
    sql=f'''
    SELECT 
        PatientID,
        StatusDesc AS 'Reassessment SPARS'
    FROM 
        tblNOMSReAssessmentDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    reassessment_SPARS = pd.read_sql(sql,conn)
    
    logger.info(f"Successfully queried tblNOMSAssessmentDetails for Reassessments entered to SPARS.")
except Exception as e:
    logger.error(f"Failed to query tblNOMSAssessmentDetails for Reassessments entered to SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)

#### Discharges 

In [15]:
try:
    sql=f'''
    SELECT 
        PatientID,
        StatusDesc AS 'Discharge SPARS'
    FROM 
        tblNOMSDischargeDetails
    WHERE
        PatientID NOT IN ({testPatients})
    ORDER BY
        PatientID
    '''
    discharge_SPARS = pd.read_sql(sql,conn)
    discharge_SPARS.drop_duplicates(inplace=True)    
    
    logger.info(f"Successfully queried tblNOMSAssessmentDetails for Discharges entered to SPARS.")
except Exception as e:
    logger.error(f"Failed to query tblNOMSAssessmentDetails for Discharges entered to SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)
    

## Merges

### Merge patient_details and encounters 

In [16]:
try:
    full_data_collection_df = patient_details_df.merge(encounter_df,
                                                       on='PatientID',
                                                       how='left')
    # drop patients without encounters
    full_data_collection_df = full_data_collection_df[full_data_collection_df['EncounterDate'].notna()]
    
    logger.info(f"Successfully merged patient deatils with encounters.")
except Exception as e:
    logger.error(f"Failed to merge patient deatils with encounters.", exc_info=True) 
    print(e)
    sys.exit(1)

### Merge in Statuses

In [17]:
try:
    full_data_collection_df = full_data_collection_df.merge(discharged_patients_df,
                                                            on = 'PatientID',
                                                            how = 'left')
    # fill nan status
    full_data_collection_df['Status'].fillna('Active',inplace=True)
    
    logger.info(f"Successfully merged in discharged patients.")
except Exception as e:
    logger.error(f"Failed to merge in discharged patients.", exc_info=True) 
    print(e)
    sys.exit(1)   

C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\829763811.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data_collection_df['Status'].fillna('Active',inplace=True)


### Merge in Baseline NOMS

In [18]:
try:
    full_data_collection_df = full_data_collection_df.merge(baseline_df,
                                                            on='PatientID',
                                                            how='left')
    full_data_collection_df['Baseline NOMS'].fillna(False,inplace=True)

    logger.info(f"Successfully merged in Baseline NOMS.")
except Exception as e:
    logger.error(f"Failed to merge in Baseline NOMS.", exc_info=True) 
    print(e)
    sys.exit(1)   

C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\3782217923.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data_collection_df['Baseline NOMS'].fillna(False,inplace=True)
C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\3782217923.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_data_collection

### Merge in Reassessment NOMS

In [19]:
try:
    full_data_collection_df = full_data_collection_df.merge(reassessment_df,
                                                            on='PatientID',
                                                            how='left')
    full_data_collection_df['6 Month Reassessment NOMS'].fillna(False,inplace=True)

    logger.info(f"Successfully merged in Reassessment NOMS.")
except Exception as e:
    logger.error(f"Failed to merge in Reassessment NOMS.", exc_info=True) 
    print(e)
    sys.exit(1) 

C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\2877868691.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data_collection_df['6 Month Reassessment NOMS'].fillna(False,inplace=True)
C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\2877868691.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_dat

### Merge in Discharge NOMS 

In [20]:
try:
    full_data_collection_df = full_data_collection_df.merge(discharge_df,
                                                            on='PatientID',
                                                            how='left')
    full_data_collection_df['Discharge NOMS'].fillna(False,inplace=True)

    logger.info(f"Successfully merged in Discharge NOMS.")
except Exception as e:
    logger.error(f"Failed to merge in Discharge NOMS.", exc_info=True) 
    print(e)
    sys.exit(1) 

C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\2464734671.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data_collection_df['Discharge NOMS'].fillna(False,inplace=True)
C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\2464734671.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_data_collectio

### Merge in SPARS Status

In [21]:
try: # add the baseline SPARS
    full_data_collection_df = full_data_collection_df.merge(baseline_SPARS,
                                                            on='PatientID',
                                                            how='left')
    logger.info(f"Successfully merged in Baseline SPARS.")
except Exception as e:
    logger.error(f"Failed to merge in Baseline SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)
    

try: # add the reassessment SPARS
    full_data_collection_df = full_data_collection_df.merge(reassessment_SPARS,
                                                        on='PatientID',
                                                        how='left')
    logger.info(f"Successfully merged in Reassessment SPARS.")
except Exception as e:
    logger.error(f"Failed to merge in Reassessment SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)


try: # add the discharge SPARS
    full_data_collection_df = full_data_collection_df.merge(discharge_SPARS,
                                                        on='PatientID',
                                                        how='left')
    logger.info(f"Successfully merged in Discharge SPARS.")
except Exception as e:
    logger.error(f"Failed to merge in Discharge SPARS.", exc_info=True) 
    print(e)
    sys.exit(1)

foo = full_data_collection_df.copy()

## Merge in New Notes

In [22]:
NoteSubTypeDict = {5: "First Attempt",
                    6: 'First Attempt',
                    7: "First Attempt"}
NoteSubTypeColumnDict = {5: 'Baseline SPARS',
                          6: "Reassessment SPARS",
                          7: "Discharge SPARS"}
try:
    sql=f'''
    SELECT 
        *
    FROM 
        tblPatientNotes
    WHERE
        NoteSubType IN (5, 6, 7)
    ORDER BY
        PatientID
    '''
    patientNotes = pd.read_sql(sql,conn)
    
    patientNotes['NoteAddedOn'] = pd.to_datetime(patientNotes['NoteAddedOn'])
    patientNotes.sort_values(by='NoteAddedOn', ascending=False)
    patientNotes['NoteSubTypeName'] = patientNotes['NoteSubType'].map(lambda subtypeID: NoteSubTypeDict[int(subtypeID)])
    patientNotes['NoteSubTypeColumn'] = patientNotes['NoteSubType'].map(lambda subtypeID: NoteSubTypeColumnDict[int(subtypeID)])
    patientNotesPivot = patientNotes.pivot(index=['PatientID'], columns = "NoteSubTypeColumn", values="NoteSubTypeName").reset_index()
    patientbaselineDict = patientNotesPivot.set_index('PatientID').to_dict()['Baseline SPARS']
    patientReassessmentDict = patientNotesPivot.set_index('PatientID').to_dict()['Reassessment SPARS']
    patientDischargeDict = patientNotesPivot.set_index('PatientID').to_dict()['Discharge SPARS']

    def checkforBaselineNotes(patientID, note):
        if note == "" or str(note) == 'nan':
            return patientbaselineDict.get(patientID, "")
        else:
            return note

    def checkforReassessmentNotes(patientID, note):
        if note == "" or str(note) == 'nan':
            return patientReassessmentDict.get(patientID, "")
        else:
            return note

    def checkforDischargeNotes(patientID, note):
        if note == "" or str(note) == 'nan':
            return patientDischargeDict.get(patientID, "")
        else:
            return note
    full_data_collection_df['Baseline SPARS'] = full_data_collection_df.apply(lambda row: checkforBaselineNotes(row['PatientID'], row['Baseline SPARS']), axis=1)
    full_data_collection_df['Reassessment SPARS'] = full_data_collection_df.apply(lambda row: checkforReassessmentNotes(row['PatientID'], row['Reassessment SPARS']), axis=1)
    full_data_collection_df['Discharge SPARS'] = full_data_collection_df.apply(lambda row: checkforDischargeNotes(row['PatientID'], row['Discharge SPARS']), axis=1)
#     logger.info(f"Successfully queried tblNotes and created columns.")
except Exception as e:
#     logger.error(f"Failed to query table notes and creat columns.", exc_info=True) 
    print(e)

## Prettify Data

In [23]:
full_data_collection_df = foo.copy()

try: # Format Names
    first_name = full_data_collection_df['FirstName'].map(lambda name: str(name).capitalize())
    last_name = full_data_collection_df['LastName'].map(lambda name: str(name).capitalize())
    full_data_collection_df['Name'] = last_name + ', ' + first_name
    full_data_collection_df.drop(columns=['FirstName','LastName'], inplace=True)
    
    logger.info(f"Successfully formated names.")
except Exception as e:
    logger.error(f"Failed to format names.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try: # Baseline Date
    full_data_collection_df['Hour'] = pd.to_datetime(full_data_collection_df['Hour'])
    full_data_collection_df['Baseline Due Date'] = full_data_collection_df['Hour'] + DateOffset(days=30)
    full_data_collection_df['Baseline Due Date'] = full_data_collection_df['Baseline Due Date'].map(lambda num: num.strftime('%m-%d-%Y'))

    logger.info(f"Successfully added Baseline Due Date.")
except Exception as e:
    logger.error(f"Failed to add Baseline Due Date.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # Baseline Warning
    full_data_collection_df['Baseline Due Date'] = pd.to_datetime(full_data_collection_df['Baseline Due Date'])
    full_data_collection_df['Baseline Warning'] = (full_data_collection_df['Baseline Due Date'] <= (pd.Timestamp.today())) & (full_data_collection_df['Baseline NOMS'] == False)
    
    logger.info(f"Successfully added Baseline Warning.")
except Exception as e:
    logger.error(f"Failed to add Baseline Warning.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # 6 Month Reassessment Date
    # we're going to set the default encounter date to the 15th of the month to match SPARS
    full_data_collection_df['Hour'] = full_data_collection_df['Hour'].apply(lambda dt: dt.replace(day=15))
    full_data_collection_df['6 Month Reassessment Date'] = full_data_collection_df['Hour'] + DateOffset(months=6)
    full_data_collection_df['6 Month Reassessment Date'] = full_data_collection_df['6 Month Reassessment Date'].map(lambda num: num.strftime('%m-%d-%Y'))

    logger.info(f"Successfully added Reassessment Date.")
except Exception as e:
    logger.error(f"Failed to add Reassessment Date.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # 6 Month Warning
    full_data_collection_df['6 Month Reassessment Date'] = pd.to_datetime(full_data_collection_df['6 Month Reassessment Date'])
    full_data_collection_df['6 Month Reassessment Warning'] = full_data_collection_df['6 Month Reassessment Date'] - DateOffset(months=1) <= pd.Timestamp.today()

    logger.info(f"Successfully added Reassessment Warning.")
except Exception as e:
    logger.error(f"Failed to add Reassessment Warning.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:    # add days since first encounter
    today = datetime.date.today()
    today = pd.to_datetime(today)
    def daysSinceFirstEncounter(row):
        date = row['EncounterDate']
        if type(date) == pd.Timestamp:
            return (today - date).days
        return np.nan
#     def daysTillBLineCompletion(row):
#         encounter_date = row['EncounterDate']
#         if isinstance(encounter_date, pd.Timestamp):
            
    def daysTillThirtyDaysAfter(row):
        encounter_date = row['EncounterDate']
        if isinstance(encounter_date, pd.Timestamp):
            # Calculate thirty days after the encounter date
            thirty_days_after = encounter_date + pd.DateOffset(days=30)
            days_till_thirty_days_after = (thirty_days_after - pd.Timestamp.today()).days
            return days_till_thirty_days_after
        return np.nan
    def daysTillSevenMonths(row):
        encounter_date = row['EncounterDate']
        if isinstance(encounter_date, pd.Timestamp):
            # Calculate seven months from the encounter date
            seven_months_later = encounter_date + pd.DateOffset(months=7)
            # Calculate the number of days between today and seven months later
            days_till_seven_months = (seven_months_later - pd.Timestamp.today()).days
            return days_till_seven_months
        return np.nan
    full_data_collection_df['Days Since Encounter'] = full_data_collection_df.apply(lambda row: daysSinceFirstEncounter(row), axis=1)  
    full_data_collection_df['Days Till Reassessment Date'] = full_data_collection_df.apply(lambda row: daysTillSevenMonths(row), axis=1)  
    full_data_collection_df['Days Till Baseline Date'] = full_data_collection_df.apply(lambda row: daysTillThirtyDaysAfter(row), axis=1)  
    logger.info(f"Successfully added Days Since First Encounter and days remaining till baseline/reassessment due dates.")
except Exception as e:
    logger.error(f"Failed to add Days Since First Encounter and days remaining till baseline/reassessment due dates.", exc_info=True) 
    print(e)
    sys.exit(1)

try:    # add days since final encounter
    def daysSinceDischarge(row):
        date = row['Final Encounter']
        if type(date) == pd.Timestamp:
            return (today - date).days
        return np.nan
    full_data_collection_df['Days Since Final Encounter'] = full_data_collection_df.apply(lambda row: daysSinceDischarge(row), axis=1)

    logger.info(f"Successfully added Days Since Final Encounter.")
except Exception as e:
    logger.error(f"Failed to add Days Since Final Encounter.", exc_info=True) 
    print(e)
    sys.exit(1)
    
try:     # find all dischareged patients and reset Status to "Discharged" 
    discharge_mask_1 = full_data_collection_df['Status'] == "Discharged"
    discharge_mask_2 = full_data_collection_df['Discharge NOMS'] == True
    discharge_mask_3 = full_data_collection_df['Discharge SPARS'] == 'Entered Into SPARS'
    completed_discharge_mask = np.logical_or(discharge_mask_1, discharge_mask_2, discharge_mask_3)
    full_data_collection_df.loc[completed_discharge_mask, 'Status'] = 'Discharged'

    logger.info(f"Successfully updated discharged statuses.")
except Exception as e:
    logger.error(f"Failed to update discharged statuses.", exc_info=True) 
    print(e)
    sys.exit(1)

In [24]:
full_data_collection_df[full_data_collection_df['PatientID'].duplicated()]

,PatientID,DOB,MRNNumber,Phone,EncounterDate,Hour,Final Encounter,Status,CreatedOn,Baseline NOMS,...,Discharge SPARS,Name,Baseline Due Date,Baseline Warning,6 Month Reassessment Date,6 Month Reassessment Warning,Days Since Encounter,Days Till Reassessment Date,Days Till Baseline Date,Days Since Final Encounter
39,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,Entered Into SPARS,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
40,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,None,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
41,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,Entered Into SPARS,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
42,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,None,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
43,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,Entered Into SPARS,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
44,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,None,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
45,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,Entered Into SPARS,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
46,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,None,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
47,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,Entered Into SPARS,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0
48,621201,1980-05-20,0000000972,718-541-4343,2023-03-12 17:30:00,2023-03-15 17:00:00,2023-03-13 10:44:00,Discharged,2023-07-20,True,...,None,"Levin, Naomi",2023-04-11,False,2023-09-15,True,686,-473,-657,685.0


In [25]:
# Reorder Columns
full_data_collection_df = full_data_collection_df[['PatientID',
                                                   'Name',
                                                   'DOB',
                                                   'Phone',
                                                   'MRNNumber',
                                                   'Status',
                                                   'EncounterDate',
                                                   'Hour',
                                                   'Days Since Encounter',
                                                   'Days Till Reassessment Date',
                                                   'Days Till Baseline Date',
                                                   'Baseline NOMS',
                                                   'Baseline Due Date',
                                                   'Baseline Warning',
                                                   'Baseline SPARS',
                                                   '6 Month Reassessment NOMS',
                                                   '6 Month Reassessment Date',
                                                   '6 Month Reassessment Warning',
                                                   'Reassessment SPARS',
                                                   'Final Encounter',
                                                   'Days Since Final Encounter',
                                                   'Discharge NOMS',
                                                   'Discharge SPARS']]
full_data_collection_df.drop_duplicates('PatientID', inplace=True)

In [26]:
full_data_collection_df[full_data_collection_df['Name'].map(lambda name: True if "Alma" in name else False)]

,PatientID,Name,DOB,Phone,MRNNumber,Status,EncounterDate,Hour,Days Since Encounter,Days Till Reassessment Date,...,Baseline Warning,Baseline SPARS,6 Month Reassessment NOMS,6 Month Reassessment Date,6 Month Reassessment Warning,Reassessment SPARS,Final Encounter,Days Since Final Encounter,Discharge NOMS,Discharge SPARS
565,630028,"Almaghrabi, Raquel",2014-11-07,347-301-1877,0000009792,Active,2025-01-07 10:00:00,2025-01-15 10:00:00,19,192,...,False,Entered Into SPARS,False,2025-07-15,False,NaN,NaT,NaN,False,NaN


## Insync Anomalies

In [27]:
# looking for anomalies, NOMS == False but SPARS == True
try:    # Baseline anomalies
    baseline_SPARS = full_data_collection_df['Baseline SPARS'] == "Entered Into SPARS"
    baseline_NOMS = full_data_collection_df['Baseline NOMS'] == True
    Insync_baseline_anomalies = ~baseline_NOMS & baseline_SPARS
    Insync_baseline_anomalies = full_data_collection_df[Insync_baseline_anomalies]
    Insync_baseline_anomalies.insert(2,'Anomaly Description', "Baseline: Has SPARS, missing NOMS")

    logger.info(f"Successfully checked for baseline Anomalies.")
except Exception as e:
    logger.error(f"Failed to check for baseline Anomalies.", exc_info=True) 
    print(e)
    
try:        # reassessment anomalies
    reassessment_SPARS = full_data_collection_df['Reassessment SPARS'] == "Entered Into SPARS"
    reassessment_NOMS = full_data_collection_df['6 Month Reassessment NOMS'] == True
    Insync_reassessment_anomalies = ~reassessment_NOMS & reassessment_SPARS
    Insync_reassessment_anomalies = full_data_collection_df[Insync_reassessment_anomalies]
    Insync_reassessment_anomalies.insert(2,'Anomaly Description', "Reassessment: Has SPARS, missing NOMS")

    logger.info(f"Successfully checked for reassessment Anomalies.")
except Exception as e:
    logger.error(f"Failed to check for reassessment Anomalies.", exc_info=True) 
    print(e)
    
try:        # discharge anomalies
    discharge_SPARS = full_data_collection_df['Discharge SPARS'] == "Entered Into SPARS"
    discharge_NOMS = full_data_collection_df['Discharge NOMS'] == True
    Insync_discharge_anomalies = ~discharge_NOMS & discharge_SPARS
    Insync_discharge_anomalies = full_data_collection_df[Insync_discharge_anomalies]
    Insync_discharge_anomalies.insert(2,'Anomaly Description', "Discharge: Has SPARS, missing NOMS")

    logger.info(f"Successfully checked for discharge Anomalies.")
except Exception as e:
    logger.error(f"Failed to check for discharge Anomalies.", exc_info=True) 
    print(e)
    
try:        # concat anomalies into their own df
    anomaly_info = ['PatientID','Anomaly Description']
    Insync_anomalies = pd.concat([Insync_baseline_anomalies[anomaly_info],
                                  Insync_reassessment_anomalies[anomaly_info],
                                  Insync_discharge_anomalies[anomaly_info]])
    
    logger.info(f"Successfully created insync anomalies df.")
except Exception as e:
    logger.error(f"Failed to create insync anomalies df.", exc_info=True) 
    print(e)
    
Insync_anomalies

,PatientID,Anomaly Description
83,622201,"Baseline: Has SPARS, missing NOMS"
17,620721,"Reassessment: Has SPARS, missing NOMS"
126,623541,"Reassessment: Has SPARS, missing NOMS"
131,623671,"Reassessment: Has SPARS, missing NOMS"


## Get Data from SPARS

In [28]:
try:     #read in SPARS excel
    SPARS_data_path = "..\Data\SPARS Download 6-24-2024.xlsx" # does it download with date in name? will break if yes
    SPARS_data_df = pd.read_excel(SPARS_data_path)

    logger.info(f"Successfully read in SPARS excel.")
except Exception as e:
    logger.error(f"Failed to read in SPARS excel.", exc_info=True) 
    print(e)
# SPARS_data_df

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\392115370.py:2: SyntaxWarning: invalid escape sequence '\D'
  SPARS_data_path = "..\Data\SPARS Download 6-24-2024.xlsx" # does it download with date in name? will break if yes


## Filter down SPARS to match Insync 

In [29]:
try:     # filter down MRN numbers
    SPARS_data_df['ClientID'] = SPARS_data_df['ClientID'].str.replace("'","")
    MRN_mask  = SPARS_data_df['ClientID'].map(lambda value: True if value[-1] == "2" else False)
    SPARS_data_df = SPARS_data_df[MRN_mask]

    logger.info(f"Successfully filtered SPARS MRNs.")
except Exception as e:
    logger.error(f"Failed to filter SPARS MRNs.", exc_info=True) 
    print(e)

In [30]:
try:    # filter first encounters after march
    march_mask = SPARS_data_df['FirstReceivedServicesDate'] > '2023-03-01'
    SPARS_data_df = SPARS_data_df[march_mask]

    logger.info(f"Successfully filtered SPARS first encounters.")
except Exception as e:
    logger.error(f"Failed to filter SPARS first encounters.", exc_info=True) 
    print(e)

In [31]:
try:    # filter out other programs
    program_mask = SPARS_data_df['ClientID'].map(lambda value: True if value[0] == "0" else False)
    SPARS_data_df = SPARS_data_df[program_mask]
    
    logger.info(f"Successfully filtered SPARS other programs.")
except Exception as e:
    logger.error(f"Failed to filter SPARS other programs.", exc_info=True) 
    print(e)

## Select data

In [32]:
columns_wanted = ['ClientID','Assessment','CalculatedInterviewDate','ConductedInterview','WhyNotConducted']
assessment_dictionary = {
    "Baseline" : 600,
    "Reassessment": 601,
    "Discharge" : 699
}
interview_not_conducted_dictionary = {
    1 : "Not able to obtain consent from proxy",
    2 : "Client was impaired or unable to provide consent",
    3 : "Client refused this interview",
    4 : "Client was not reached for interview",
    5 : "Client refused all interviews",
    -1 : "Client was interviewed"
}

In [33]:
try:
    SPARS_assessments = pd.DataFrame(columns = columns_wanted) # create df for the desired data 
    for assessment in assessment_dictionary:
        assessment_df = SPARS_data_df[SPARS_data_df['Assessment'] == assessment_dictionary.get(assessment)] # copy in all data for current assessment type
        assessment_df = assessment_df.drop_duplicates(['ClientID']) # get rid of duplicates on name (shouldn't be needed but there was a duplicated baseline)
        assessment_df = assessment_df[columns_wanted] # remove all unneeded columns 
        assessment_df['Assessment'] = assessment # set assessment type values
        SPARS_assessments = pd.concat([SPARS_assessments[columns_wanted], # add current assessment data to all assessments' data
                                       assessment_df[columns_wanted]])
    SPARS_assessments.rename(columns={"ClientID": "MRN Number"},inplace=True)
    SPARS_assessments
    
    logger.info(f"Successfully selected useful SPARS data.")
except Exception as e:
    logger.error(f"Failed to select useful SPARS data.", exc_info=True) 
    print(e)
SPARS_assessments    

C:\Users\ylen\AppData\Local\Temp\ipykernel_20904\2836527459.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  SPARS_assessments = pd.concat([SPARS_assessments[columns_wanted], # add current assessment data to all assessments' data


,MRN Number,Assessment,CalculatedInterviewDate,ConductedInterview,WhyNotConducted
0,0000001222,Baseline,2023-03-15,0,4
1,0000000942,Baseline,2023-03-15,0,2
2,0000002452,Baseline,2023-03-15,0,3
3,0000001922,Baseline,2023-03-15,0,2
4,0000000312,Baseline,2023-03-15,0,3
...,...,...,...,...,...
970,0000005052,Discharge,2024-07-15,1,-1
971,0000005212,Discharge,2024-08-15,1,-1
978,0000000592,Discharge,2024-09-15,1,-1
979,0000002162,Discharge,2024-08-15,1,-1


## Replace Values

In [34]:
try:    # replace values using dictionaries from SPARS
    for reason in interview_not_conducted_dictionary:
        SPARS_assessments['WhyNotConducted'] = SPARS_assessments['WhyNotConducted'].replace(reason, interview_not_conducted_dictionary.get(reason))

    logger.info(f"Successfully replaced values for SPARS not conducted.")
except Exception as e:
    logger.error(f"Failed to replace values for SPARS not conducted.", exc_info=True) 
    print(e)        

## Compare with Insync

In [35]:
try:    # which MRNs exist only Insync or SPARS but not both
    full_data_collection_df['SPARS MRN'] = full_data_collection_df['MRNNumber'].isin(SPARS_assessments['MRN Number'].drop_duplicates())
    SPARS_assessments['Insync MRN'] = SPARS_assessments['MRN Number'].isin(full_data_collection_df['MRNNumber'].drop_duplicates())
    
    logger.info(f"Successfully compared SPARS and Insync MRNs.")
except Exception as e:
    logger.error(f"Failed to compare SPARS and Insync MRNs.", exc_info=True) 
    print(e)

## Get download Date

In [36]:
try:    # add download date
    file_stat = os.stat(SPARS_data_path)

    # Check if st_birthtime attribute is available
    if hasattr(file_stat, 'st_birthtime'):
        # Use st_birthtime for the creation time
        creation_time = file_stat.st_birthtime
    else:
        # Fallback to st_mtime for the last modification time
        creation_time = file_stat.st_mtime
    
    # Convert the creation time to a datetime object
    creation_datetime = datetime.datetime.fromtimestamp(creation_time)
    # creation_datetime = datetime.fromtimestamp(creation_time)
    
    # Format the datetime object into MM/DD/YYYY
    creation_date_formatted = creation_datetime.strftime('%m/%d/%Y')
    
    SPARS_assessments['Download Date'] = creation_date_formatted

    logger.info(f"Successfully added SPARS download date.")
except Exception as e:
    logger.error(f"Failed to add SPARS download date.", exc_info=True) 
    print(e)

## Push to DB

In [37]:
# with pd.ExcelWriter(r"../data/NOMS Survey Completion.xlsx") as writer: 
#     full_data_collection_df.to_excel(writer, sheet_name="Patient Data",index = False)
#     consent_df.to_excel(writer, sheet_name="Consent Data",index = False)

In [38]:
# ", ".join([item + " " + str(full_data_collection_df[item].dtype) for item in full_data_collection_df.columns])

### Push full_data_collection (Insync data)

In [39]:
table_name = "ptNOMS_Completion"
# try:
#     c = conn.cursor()
#     c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER,
#                                                            Name TEXT,
#                                                            DOB TEXT,
#                                                            Phone, TEXT
#                                                            MRNNumber TEXT,
#                                                            Status TEXT,
#                                                            EncounterDate TEXT,
#                                                            Hour TEXT,
#                                                            DaysSinceEncounter INTEGER,
#                                                            DaysTillBaseline INTEGER,
#                                                            DaysTillReassessment INTEGER
#                                                            BaselineNOMS TEXT,
#                                                            BaselineDueDate TEXT,
#                                                            BaselineWarning TEXT,
#                                                            Baseline SPARS TEXT,
#                                                            _6MonthReassessmentNOMS TEXT,
#                                                            _6MonthReassessmentDate TEXT,
#                                                            _6MonthReassessmentWarning TEXT,
#                                                            ReassessmentSPARS TEXT,
#                                                            FinalEncounter TEXT,
#                                                            DaysSinceFinalEncounter INTEGER,
#                                                            DischargeNOMS TEXT,
#                                                            DischargeSPARS TEXT,
#                                                            SPARS MRN TEXT)''')
#     conn.commit()
#     logger.info(f"Successfully created {table_name}.")
# except Exception as e:
#     logger.error(f"Failed to create {table_name}.", exc_info=True) 
#     print(e)
    
try:
    full_data_collection_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)

### Push consent_df  

In [40]:
# ", ".join([item + " " + str(consent_df[item].dtype) for item in consent_df.columns])

In [41]:
table_name = "ptNOMS_Consent"
# try:
#     c = conn.cursor()
#     c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER,
#                                                            Consent TEXT,
#                                                            Reason TEXT)''')
#     conn.commit()
#     logger.info(f"Successfully created {table_name}.")
# except Exception as e:
#     logger.error(f"Failed to create {table_name}.", exc_info=True) 
#     print(e)
    
try:
    consent_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)

### Push Insync Anomalies

In [42]:
table_name = "ptNOMS_Insync_Anomalies"

# try:
#     c = conn.cursor()
#     c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (Patient ID INTEGER,
#                                                            Anomaly Description TEXT)''')
#     conn.commit()
#     logger.info(f"Successfully created {table_name}.")
# except Exception as e:
#     logger.error(f"Failed to create {table_name}.", exc_info=True) 
#     print(e)
    
try:
    Insync_anomalies.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)

### Push SPARS data

In [43]:
SPARS_assessments.columns.tolist()

['MRN Number',
 'Assessment',
 'CalculatedInterviewDate',
 'ConductedInterview',
 'WhyNotConducted',
 'Insync MRN',
 'Download Date']

In [44]:
table_name = "ptNOMS_SPARS_Data"

# try:
#     c = conn.cursor()
#     c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (MRN Number TEXT,
#                                                            Assessment Type TEXT,
#                                                            Interview Conducted INTEGER,
#                                                            WhyNotConducted TEXT,
#                                                            Insync MRN TEXT,
#                                                            Download Date TEXT)''')
#     conn.commit()
#     logger.info(f"Successfully created {table_name}.")
# except Exception as e:
#     logger.error(f"Failed to create {table_name}.", exc_info=True) 
#     print(e)
    
try:
    SPARS_assessments.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)

In [45]:
conn.dispose()

In [46]:
# full_data_collection_df.to_clipboard()